In [5]:
import os
from concurrent.futures import ThreadPoolExecutor

In [6]:
fmask_path = "/home/mxh/anaconda3/envs/fmask/bin/"
gdal_path = "/home/mxh/anaconda3/envs/fmask/bin/"
project_path = "/home/mxh/RS-Net/"

In [7]:
def _make_toa_and_png(product):
    print("Processing product:", product)
    
    # Save indivual bands temporarily
    for b in [1, 2, 3, 4, 5, 6, 7, 9, 10, 11]:
        !{gdal_path}gdal_translate -b {b} {data_path[:-1]}/{product[0:24]}_data.tif {data_output_path+product[0:25]}B{b}.tif

    # Run Fmask
    !{gdal_path}gdal_merge.py -separate -of HFA -co COMPRESSED=YES -o {data_output_path[:-1]}/{product[0:24]}_ref.img {data_output_path[:-1]}/LC8*_B[1-7,9].tif        
    !{gdal_path}gdal_merge.py -separate -of HFA -co COMPRESSED=YES -o {data_output_path[:-1]}/{product[0:24]}_thermal.img {data_output_path[:-1]}/LC8*_B1[0,1].tif

    !{fmask_path}fmask_usgsLandsatMakeAnglesImage.py -m {data_path[:-1]}/{product[0:15]}*_mtl.txt -t {data_output_path[:-1]}/{product[0:24]}_ref.img -o {data_output_path[:-1]}/{product[0:24]}_angles.img
    !{fmask_path}fmask_usgsLandsatSaturationMask.py -i {data_output_path[:-1]}/{product[0:24]}_ref.img -m {data_path[:-1]}/{product[0:15]}*_mtl.txt -o {data_output_path[:-1]}/{product[0:24]}_saturationmask.img
    !{fmask_path}fmask_usgsLandsatTOA.py -i {data_output_path[:-1]}/{product[0:24]}_ref.img -m {data_path[:-1]}/{product[0:15]}*_mtl.txt -z {data_output_path[:-1]}/{product[0:24]}_angles.img -o {data_output_path[:-1]}/{product[0:24]}_toa.img
    !{fmask_path}fmask_usgsLandsatStacked.py -t {data_output_path[:-1]}/{product[0:24]}_thermal.img -a {data_output_path[:-1]}/{product[0:24]}_toa.img -m {data_path[:-1]}/{product[0:15]}*_mtl.txt -z {data_output_path[:-1]}/{product[0:24]}_angles.img -s {data_output_path[:-1]}/{product[0:24]}_saturationmask.img -o {data_output_path[:-1]}/{product[0:24]}_fmask.tif --cloudbufferdistance 0 --shadowbufferdistance 0
    
    # Translate to .tif file
    !{gdal_path}gdal_translate -of Gtiff {data_output_path[:-1]}/{product[0:24]}_toa.img {data_processed_path[:-1]}/{product[0:24]}_toa.TIF
    # Translate to .png file
    !{gdal_path}gdal_translate -of PNG {data_output_path[:-1]}/{product[0:24]}_fmask.tif {data_output_path[:-1]}/{product[0:24]}_fmask.png 
    
    # Delete temp files
    !rm {data_output_path}{product[0:24]}*.tif {data_output_path}{product[0:24]}*.img {data_output_path}{product[0:24]}*.xml

In [15]:
data_path = project_path + "data/raw/SPARCS_dataset/l8cloudmasks/sending/"
data_output_path = project_path + "data/output/SPARCS/"
data_processed_path = project_path + "data/processed/SPARCS_TOA/"

products = sorted(os.listdir(data_path))
products = [p for p in products if 'data.tif' in p]

# with ThreadPoolExecutor(max_workers=1) as tpe:
    # timeout 10: max wait 60s on one product calculation
    # results = tpe.map(_make_toa_and_png, products, timeout=60)

#lazy eval
results = map(_make_toa_and_png, products)

In [9]:
for r in results:
    print(r)

Processing product: LC80010812013365LGN00_18_data.tif
Warning 1: /home/mxh/RS-Net/data/raw/SPARCS_dataset/l8cloudmasks/sending/LC80010812013365LGN00_18_data.tif: TIFFReadDirectory:Sum of Photometric type-related color channels and ExtraSamples doesn't match SamplesPerPixel. Defining non-color channels as ExtraSamples.
Input file size is 1000, 1000
Warning 1: TIFFReadDirectory:Sum of Photometric type-related color channels and ExtraSamples doesn't match SamplesPerPixel. Defining non-color channels as ExtraSamples.
0...10...20...30...40...50...60...70...80...90...100 - done.
Warning 1: /home/mxh/RS-Net/data/raw/SPARCS_dataset/l8cloudmasks/sending/LC80010812013365LGN00_18_data.tif: TIFFReadDirectory:Sum of Photometric type-related color channels and ExtraSamples doesn't match SamplesPerPixel. Defining non-color channels as ExtraSamples.
Input file size is 1000, 1000
Warning 1: TIFFReadDirectory:Sum of Photometric type-related color channels and ExtraSamples doesn't match SamplesPerPixel. 

In [14]:
# test if all products have been converted

for p in products:
    if not os.path.exists(data_processed_path + p[0:24] + "_toa.TIF"):
        print("toa Tif does not exist for product: ", p)
    if not os.path.exists(data_output_path + p[0:24] + "_fmask.png"):
        print("fmask PNG does not exist for product: ", p)

In [18]:
#for p in products:
#    !mv {data_output_path + p[0:24] + "_toa.TIF"} {data_processed_path + p[0:24] + "_toa.TIF"}